# Predicting wind energy production with Machine Learning

Arturo Prieto Tirado and Pere Fuster Escrivà

In [26]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import train_test_split # 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
import sklearn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn import tree

Load the dataset, delete month, steps, day and hour and insert NA

In [4]:
my_NIA = 100441384 # NIA Pere Fuster 
np.random.seed(my_NIA)

data = pd.read_pickle('wind_pickle.pickle')
data = data.drop(['month','steps','day','hour'], axis=1)

grey = data.drop(['year','energy'], axis=1)
grey = grey.sample(frac=0.10, axis='columns')

data.shape
grey.shape

for col in grey.columns:
    grey.loc[grey.sample(frac = 0.05).index, col] = np.nan

grey.isna().sum()/grey.shape[0] # checking, my nigga

data[grey.columns] = grey

data.isna().sum()/data.shape[0] # double-checking, my nigga



energy       0.000000
year         0.000000
p54.162.1    0.000000
p54.162.2    0.050025
p54.162.3    0.000000
               ...   
v100.21      0.000000
v100.22      0.000000
v100.23      0.000000
v100.24      0.050025
v100.25      0.000000
Length: 552, dtype: float64

Create the training, validation and testing datasets in chronological order

In [5]:
data.year.min() # checking years ranges 
data.year.max()

#Train partition: data belonging to 2005 and 2006.
# Validation partition (inner): 2007 and 2008 data.
# Test partition (outer): 2009 and 2010.

#train = data[(data['year'] < 2009)]
train = data[(data['year'] < 2007)]
validation = data[(data['year'] >= 2007) & (data['year'] < 2009)]
holdout = data[(data['year'] >= 2009) & (data['year'] < 2010)]



Column year should be removed now and data should be transformed to Numpy matrices, so that they can be used by scikit-learn

In [6]:
train = train.drop(['year'], axis=1)
#training = training.drop(['year'], axis=1)
validation = validation.drop(['year'], axis=1)
holdout = holdout.drop(['year'], axis=1)

# Model selection and Hyperparameter tuning

The code belows includes all the steps related to data partition, which will be necessary to evaluate the models and to be able to improve their performance with parameter tuning. 
First, we remove the response, energy, from the training and testing set and store it as a response y.

In [7]:
X_train = train.drop(['energy'], axis=1)
y_train = train.energy

X_test = train.drop(['energy'], axis=1)
y_test = train.energy




Now, we define a train validation grid search

In [8]:
from sklearn.model_selection import PredefinedSplit
import numpy as np

# Defining a fixed train/validation grid-search
# -1 means training, 0 means validation

validation_indices = np.zeros(X_train.shape[0])
validation_indices[:round(2/3*X_train.shape[0])] = -1
np.random.seed(0) # This is for reproducibility
validation_indices = np.random.permutation(validation_indices)
tr_val_partition = PredefinedSplit(validation_indices)

Since KNN and SVM need the same preprocessing, this is done at once


# KNN


Let's start with KNN with default hyperparameters. We will build a pipeline that applies standarisation, imputes the missing values and does feature selection with classificator, the KNN.

In [9]:

# Standardisation needed for KNN
MinMax = sklearn.preprocessing.MinMaxScaler()

# Replace NAs 
imputer = sklearn.impute.SimpleImputer()

# Feature selection 
selector = sklearn.feature_selection.SelectKBest()

# Classificator 
knn = KNeighborsRegressor()

# Building the pipeline 
reg = Pipeline([
('minmax', MinMax), 
('impute', imputer),
('select', selector),
('knn_regression', knn)])


Once we have the pipeline, we can use the fit() function on it with the training dataset and response and afterwards make a prediction with the predict() function and the testing dataset.

In [11]:
#fitting the model
reg.fit(X_train, y_train)
#making a prediction
y_test_pred = reg.predict(X_test)
#printing MAE
print(metrics.mean_absolute_error(y_test, y_test_pred))

270.30133227848097


We can see that we get a Mean Absolute Error of 270.3, which doesn't say much on itself, but it is useful to compare between models.

Next step is using KNN with hyperparameter tuning. We introduce the pipeline in the same way as before

In [12]:
### KNN and SVM PIPELINE 

# Standardisation 
MinMax = sklearn.preprocessing.MinMaxScaler() # SVMs assume that the data it works with is in a standard range

# NAs Action 
imputer = sklearn.impute.SimpleImputer()

# Feature selection 
selector = sklearn.feature_selection.SelectKBest() # Select the most important 3 
# Classificator 
knn = KNeighborsRegressor()

# Building the pipeline 

reg = Pipeline([
('minmax', MinMax), 
('impute', imputer),
('select', selector),
('knn_regression', knn)])


But now we add hyper-parameter search space so that it can impute the NA using the median or the mean; work with 1 up to 12 neighbors with uniform or weighted distance. This way, the algorithm will find the best choice for all of them

In [13]:
# Defining hyper-parameter (search) space


param_grid = {
'impute__strategy': ['mean', 'median'],
'knn_regression__n_neighbors': range(1,12),
'knn_regression__weights': ['uniform', 'distance']
} 

We also need to specify the search method for the cross validation grid that will be used in the hyperparameter tuning. We use the negative mean absolute error and the validation partition. Also, n_jobs allows us to work in parallel to be faster. Since our computers have 8 cores, we will use 6.

In [14]:
# Defining Search Method, evaluation method and performance measure
reg_grid = RandomizedSearchCV(reg, param_grid, scoring='neg_mean_absolute_error', cv=tr_val_partition, n_jobs=6, verbose=1) 

We can finally fit our model using the training.

In [15]:
#fitting the model
reg_grid = reg_grid.fit(X_train, y_train)

Fitting 1 folds for each of 10 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:    2.7s finished


And make predictions using predict() and the testing dataset.

In [24]:
# The tuned method can be used for making predictions, just as any fit machine learning method
y_test_pred = reg_grid.predict(X_test)

print(reg_grid.best_params_)
print(-reg_grid.best_score_)

print(metrics.mean_absolute_error(y_test, y_test_pred))

{'knn_regression__weights': 'distance', 'knn_regression__n_neighbors': 11, 'impute__strategy': 'median'}
405.01283240149667
0.0


Por qué dan diferentes?? Se supone que la best score es el mejor negative mean absolute error, que es lo mismo que hace metrics.mean absolute error (no usar metrics.mean_absolute_error?). Además, debería dar en MAE más bajo que la parte sin hyperparameter tuning, y sale más grande.

# SVM

The goal of an SVM is to create a flat boundary, called hyperplane, which leads to fairly homogeneous partitions of data on either side.

Therefore these are the hyperameters of these applications: 

- C is the penalty parameter, it models the importance of missclasification
- Kernel is the kernel function used to map the data to a higher dimension space
- epsilon is the margin of tolerance where to penalty is given to errors

Once the dimension have been increased to allow for a flat linear separation.

First we do SVM with default hyperparamters. We define the pipeline that scales, imputes NA and does feature selection with default hyperparameters analogously as we did for KNN.

In [27]:
# Standardisation 
MinMax = sklearn.preprocessing.MinMaxScaler()

# NAs Action 
imputer = sklearn.impute.SimpleImputer()

# Feature selection 
selector = sklearn.feature_selection.SelectKBest() # Select the most important 3 

# Classificator 
svm = SVR()

# Building the pipeline 
reg = Pipeline([
('minmax', MinMax), 
('impute', imputer),
('select', selector),
('supportVM', svm)])

Now we proceed to fit the model using the training dataset

In [28]:
reg.fit(X_train, y_train)

Pipeline(steps=[('minmax', MinMaxScaler()), ('impute', SimpleImputer()),
                ('select', SelectKBest()), ('supportVM', SVR())])

Finally, predict and check its negative MAE

In [29]:
y_test_pred = reg.predict(X_test)
print(metrics.mean_absolute_error(y_test, y_test_pred))

452.5666004703861


We get a MAE of 452.57, let's now use SVM with hyperparameter tuning and compare.
The pipeline is done in the same way as well.

In [31]:
# Standardisation 
MinMax = sklearn.preprocessing.MinMaxScaler()

# NAs Action 
imputer = sklearn.impute.SimpleImputer()

# Feature selection 
selector = sklearn.feature_selection.SelectKBest() # Select the most important 3 

# Classificator 
svm = SVR()

# Building the pipeline 
reg = Pipeline([
('minmax', MinMax), 
('impute', imputer),
('select', selector),
('supportVM', svm)])



Now it's time to define the hyper-parameter space where to search. The hyperparameters are the way to replace NAs: mean or median; the penalty parameter; the type of kernel function: linear, polynomical, radial basis function (rfb) or a sigmoid and the epsilon.

In [32]:
param_grid = {
'impute__strategy': ['mean', 'median'],
'supportVM__C': list(np.linspace(0.1, 50, 500)),
'supportVM__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
'supportVM__epsilon': list(np.linspace(0.001, 1, 1000))
} 

We also need to specify the search method for the cross validation grid that will be used in the hyperparameter tuning. We use the negative mean absolute error and the validation partition in parallel as done for KNN.

In [33]:
# Defining Search Method, evaluation method and performance measure
reg_grid = RandomizedSearchCV(reg, param_grid, scoring='neg_mean_absolute_error', cv=tr_val_partition, n_jobs=7, verbose=1) 

Now we can proceed to fit the model

In [34]:
reg_grid = reg_grid.fit(X_train, y_train)

Fitting 1 folds for each of 10 candidates, totalling 10 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    3.0s remaining:    0.7s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    3.8s finished


Finally, let's make predictions with the testing dataset and analyze the power of the model with the MAE

In [35]:
y_test_pred = reg_grid.predict(X_test)

reg_grid.best_params_
reg_grid.best_score_

print(metrics.mean_absolute_error(y_test, y_test_pred))

338.0660757653248


We get a MAE of 338, way better than the SVM with default hyperparameters.

# Regression Trees

The last model we will work with are regression trees. These models don't need scaling and can work with NAs, so we define a simpler pipeline:

In [37]:
# Feature selection 
selector = sklearn.feature_selection.SelectKBest() # Select the most important 

# Classificator 
trees=tree.DecisionTreeClassifier()

# Building the pipeline 
reg = Pipeline([
('select', selector),
('decisiontree', trees)])

Decision trees split into values, so we just need to assign to the NAs a value that no other one has.

Now we can fit the model using the training dataset

In [38]:
reg.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

And predict using the testing dataset

In [ ]:
y_test_pred = reg.predict(X_test)
print(metrics.mean_absolute_error(y_test, y_test_pred))

We get a MAE of:

Now let's work with hyperparameter tuning. We can modify the depth of the tree and the number of leaves

# Atribute selection using KNN

Only feature selection

In [43]:
# Standardisation 
MinMax = sklearn.preprocessing.MinMaxScaler() # SVMs assume that the data it works with is in a standard range

# NAs Action 
imputer = sklearn.impute.SimpleImputer(strategy='mean')

# Feature selection 
selector = sklearn.feature_selection.SelectKBest() # Select the most important 3 
# Classificator 
knn = KNeighborsRegressor()

# Building the pipeline 

reg = Pipeline([
('minmax', MinMax), 
('impute', imputer),
('select', selector),
('knn_regression', knn)])

# Defining hyper-parameter (serach) space
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

param_grid = {
'select__k':  range(1,30),
'knn_regression__n_neighbors': range(1,12),
'knn_regression__weights': ['uniform', 'distance']
} 

# Defining Search Method, evaluation method and performance measure
reg_grid = GridSearchCV(reg, param_grid, scoring='neg_mean_absolute_error', cv=tr_val_partition , n_jobs=7, verbose=1) 

reg_grid = reg_grid.fit(X_train, y_train)

# The tuned method can be used for making predictions, just as any fit machine learning method
y_test_pred = reg_grid.predict(X_test)

print(reg_grid.best_params_)
print(reg_grid.best_score_)

print(metrics.mean_absolute_error(y_test, y_test_pred))

Fitting 1 folds for each of 638 candidates, totalling 638 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    2.3s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:   11.4s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:   26.0s
[Parallel(n_jobs=7)]: Done 638 out of 638 | elapsed:   39.7s finished


{'knn_regression__n_neighbors': 11, 'knn_regression__weights': 'uniform', 'select__k': 28}
-316.0417901434272
287.6881199654776


Only PCA

In [42]:
# Standardisation 
MinMax = sklearn.preprocessing.MinMaxScaler() # SVMs assume that the data it works with is in a standard range

# NAs Action 
imputer = sklearn.impute.SimpleImputer(strategy='mean')

# Classificator 
knn = KNeighborsRegressor()

# Attribute tranformation with PCA 
from sklearn.decomposition import PCA
pca = PCA()

# Building the pipeline 

reg = Pipeline([
('minmax', MinMax), 
('impute', imputer),
("features", pca), 
('knn_regression', knn)])

# Defining hyper-parameter (serach) space
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

param_grid = {
'features__n_components':  range(1,6),
'knn_regression__n_neighbors': range(1,12),
'knn_regression__weights': ['uniform', 'distance']
} 

# Defining Search Method, evaluation method and performance measure
reg_grid = GridSearchCV(reg, param_grid, scoring='neg_mean_absolute_error', cv=tr_val_partition , n_jobs=7, verbose=1) 

reg_grid = reg_grid.fit(X_train, y_train)

# The tuned method can be used for making predictions, just as any fit machine learning method
y_test_pred = reg_grid.predict(X_test)

print(reg_grid.best_params_)
print(-reg_grid.best_score_)


Fitting 1 folds for each of 110 candidates, totalling 110 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    2.3s
[Parallel(n_jobs=7)]: Done 110 out of 110 | elapsed:    6.3s finished


{'features__n_components': 5, 'knn_regression__n_neighbors': 9, 'knn_regression__weights': 'distance'}
289.0610276199887


Both

In [44]:
# Standardisation 
MinMax = sklearn.preprocessing.MinMaxScaler() # SVMs assume that the data it works with is in a standard range

# NAs Action 
imputer = sklearn.impute.SimpleImputer(strategy='mean')

# Feature selection 
selector = sklearn.feature_selection.SelectKBest() # Select the most important 3 
# Classificator 
knn = KNeighborsRegressor()

# Attribute tranformation with PCA 
from sklearn.decomposition import PCA
pca = PCA()

# Feature Union 

combined_features = FeatureUnion([("pca", pca),
("select", selector)])

# Building the pipeline 
reg = Pipeline([
('minmax', MinMax), 
('impute', imputer),
("features", combined_features), 
('knn_regression', knn)])

# Defining hyper-parameter (serach) space
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

param_grid = {
'features__k':  range(1,30),
'features__n_components':  range(1,6),
'knn_regression__n_neighbors': range(1,12),
'knn_regression__weights': ['uniform', 'distance']
} 

# Defining Search Method, evaluation method and performance measure
reg_grid = GridSearchCV(reg, param_grid, scoring='neg_mean_absolute_error', cv=tr_val_partition , n_jobs=7, verbose=1) 

reg_grid = reg_grid.fit(X_train, y_train)

# The tuned method can be used for making predictions, just as any fit machine learning method
y_test_pred = reg_grid.predict(X_test)

reg_grid.best_params_
reg_grid.best_score_

print(metrics.mean_absolute_error(y_test, y_test_pred))

Fitting 1 folds for each of 3190 candidates, totalling 3190 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


ValueError: Invalid parameter k for estimator FeatureUnion(transformer_list=[('pca', PCA()), ('select', SelectKBest())]). Check the list of available parameters with `estimator.get_params().keys()`.

# Discussion and Conclusions